In [1]:
# !yt-dlp "https://www.youtube.com/watch?v=HFjJvWPwuJo&t=510s"

In [2]:
#!ffmpeg -i video.webm audio.mp3

In [3]:
import IPython.display as display
display.Audio("small.mp3", autoplay=True)

In [4]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import librosa

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

audio, rate = librosa.load("small.mp3", sr = 16000)

# load dummy dataset and read soundfiles
#ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

# tokenize
input_values = processor(audio, return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)

/home/iuayala/.virtualenvs/translation/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/iuayala/.virtualenvs/translation/lib/python3.8/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [5]:
print(transcription)

["A NICE LITTLE QUALITY OF LIFE THING BUT IT'S GOIN TO COST YOU ANOTHER TWENTY FIVE DOLLARS FOR THAT FUNCTIONALITY AT THE TIME OF WRITING AND IT DOESN'T SOLVE THE EVERY HERO NEEDS A DIFFERENT BINDING PROBLEM ONLY THE I PLAY A LOT OF DIFFERENT GAMES ONE BUT HEY AT LEAST IT'S COME"]


In [12]:
from transformers import pipeline

generator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
text_es = generator(transcription)
text_es

[{'translation_text': 'Una buena calidad de vida, pero te va a costar otra veintena de dólares por esa diversión en el momento de la escritura y no resuelve el problema de cada héroe que necesita una pareja diferente, sólo que yo juego un montón de juegos diferentes, pero al menos ha llegado.'}]

In [ ]:
# Narakeet seems good